<a href="https://colab.research.google.com/github/babypotatotang/Introduction-to-DeepLearning/blob/main/8.%20%EC%9E%90%EC%97%B0%EC%96%B4%20%EC%B2%98%EB%A6%AC%EC%9D%98%20%EC%A0%84%EC%B2%98%EB%A6%AC/04_%EC%98%81%EC%96%B4%20%ED%95%9C%EA%B5%AD%EC%96%B4%20Word2Vec%20%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* `gensim` 패키지에서 제공하는 이미 구현된 Word2Vec을 사용하여 영어와 한국어 데이터를 학습함. 

# **1. 영어 Word2Vec 만들기**
--- 
* 파이썬은 gensim 패키지에서 Word2Vec을 지원하고 있어, 손쉽게 단어를 임베딩 벡터로 변환시킬 수 있음. 영어로 된 코퍼스를 다운받아 전처리를 수행하고 전처리한 데이터를 바탕으로 Word2Vec 작업을 진행함. 

In [ ]:
import re 
import urllib.request
import zipfile
from lxml import etree
from nltk.tokenize import word_tokenize, sent_tokenize

## 1) 훈련데이터 이해하기
* 훈련데이터파일은 xml 문법으로 작성되어 있어 자연어를 얻기 위해서는 전처리가 필요함. 얻고자 하는 실질적 데이터는 영어 문장으로만 구성된 내용을 담고 있은 `<content>`와 `</content>` 사이의 내용임을 유의

In [ ]:
# 데이터 다운로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/ukairia777/tensorflow-nlp-tutorial/main/09.%20Word%20Embedding/dataset/ted_en-20160408.xml", filename="ted_en-20160408.xml")

## 2) 훈련데이터 전처리하기 

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
targetXML = open('ted_en-20160408.xml', 'r', encoding = 'UTF8')
target_text = etree.parse(targetXML)
target_text

In [ ]:
# xml로부터 content 내용만 가져옴
parse_text = '\n'.join(target_text.xpath('//content/text()'))
parse_text

In [ ]:
# 정규 표현식의 sub 모듈을 통해 content 중간에 등장하는 (Audio), (Laughter) 등의 배경음 부분 제거
# 해당 코드는 괄호로 구성된 내용을 제거
content_text = re.sub(r'\([^)]*\)', '', parse_text)

In [ ]:
len(content_text)

In [ ]:
# 입력 코퍼스에 대해서 NLTK를 이용하며 문장 토큰화 수행
sent_text = sent_tokenize(content_text)

In [ ]:
# 각 문장에 대해서 구두점을 제거하고, 대문자를 소문자로 변환 
normalized_text = []
for string in sent_text:
    tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
    normalized_text.append(tokens)

In [ ]:
result = [word_tokenize(sentence) for sentence in normalized_text]

In [ ]:
print(f'총 샘플의 개수 : {len(result)}')

In [ ]:
for line in result[:3]:
    print(line)

## **3) Word2Vec 훈련하기**

In [ ]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

In [ ]:
model = Word2Vec(sentences = result, size = 100, window = 5, min_count = 5, workers = 4, sg = 0)

* size = 워드 벡터의 특징 값, 즉 임베딩 된 벡터의 차원
* window = 컨텍스트 윈도우 크기
* min_count = 단어 최소 빈도 수 제한(빈도가 적은 단어들은 학습하지 않음) 
* workers = 학습을 위한 프로세스 수 
* sg = 0은 CBOW, 1은 skip gram 

In [ ]:
# 입력 단어와 가장 유사한 단어를 출력하는 함수 
model_result = model.wv.most_similar("man")
print(model_result)

## **4) Word2Vec 모델 저장하고 로드하기**

In [ ]:
model.wv.save_word2vec_format('eng_w2v') # 모델 저장 
loaded_model = KeyedVectors.load_word2vec_format("eng_w2v") # 모델 로드

In [ ]:
model_result = loaded_model.most_similar("korea")
print(model_result)

# **2. 한국어 Word2Vec 만들기(네이버 영화 리뷰)**

In [ ]:
!pip install konlpy

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import urllib.request
from gensim.models.word2vec import Word2Vec
from konlpy.tag import Okt

In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt", filename="ratings.txt")

In [ ]:
train_data = pd.read_table("ratings.txt")

In [ ]:
train_data[:5]

In [ ]:
print(train_data.isnull().values.any()) # Null값 확인 

In [ ]:
# 결측값이 존재하는 행 제거 
train_data = train_data.dropna(how = 'any') # Null 값이 존재하는 행 제거
print(train_data.isnull().values.any()) # Null 값이 존재하는지 확인

In [ ]:
print(len(train_data))

In [ ]:
# 한글 외 문자 제거
train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")

In [ ]:
train_data[:5] # 상위 5개 출력

In [ ]:
# 불용어 정의
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

In [ ]:
okt = Okt()

In [ ]:
from tqdm import tqdm

tokenized_data = []

for sentence in tqdm(train_data['document']):
    tokenized_sentence = okt.morphs(sentence, stem = True) # 토큰화 
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거 
    tokenized_data.append(stopwords_removed_sentence)

In [ ]:
print(f"리뷰의 최대 길이: {max(len(review) for review in tokenized_data)}")
print(f"리뷰의 평균 길이: {sum(map(len, tokenized_data))/ len(tokenized_data)}")

In [ ]:
from gensim.models import Word2Vec

# 토큰화된 영화리뷰 데이터를 학습함 
model = Word2Vec(sentences = tokenized_data, size = 100, window = 5, min_count = 5, workers = 4, sg = 0)

In [ ]:
model.wv.vectors.shape

In [ ]:
print(model.wv.most_similar("최민식"))

In [ ]:
print(model.wv.most_similar("히어로"))

In [ ]:
model.wv.save_word2vec_format('ko_w2v')

# **3. Pre-Trained Word2Vec embedding**
* 위키피디아 등 방대한 데이터로 사전 훈련된 워드 임베딩을 가지고 해당 벡터들의 값을 원하는 대로 사용할 수 있음. 
* 구글의 경우 사전 훈련된 3백만개의 Word2Vec 단어 벡터를 제공하는데, 이 벡터의 차원은 300임. gensim을 통해서 이 모델을 불러올 수 있음. 
    * 다운로드 경로) https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit

In [ ]:
import gensim
import urllib.request

In [ ]:
# 구글의 사전 훈련된 Word2Vec 모델을 로드.
# 저장용량은 3G 
urllib.request.urlretrieve("https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz", \
                           filename="GoogleNews-vectors-negative300.bin.gz")
word2vec_model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)

* Word2Vec 모델은 자연어 처리에서 단어를 밀집 벡터로 만들어주는 단어 임베딩 방법론임. 동시에 추천시스템에도 자주 사용되는데, 적당하게 데이터를 나열해주면 Word2Vec은 위치가 근접한 데이터를 유사도가 높은 벡터로 만들어주는 점에서 사용되며 `item2vec`이라는 이름의 아이디어로 불림. 